In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Загрузка данных
print("📊 Загрузка данных...")
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

# Предварительный анализ данных
print("\n Анализ данных:")
print(f"Размер тренировочных данных: {train_df.shape}")
print(f"Размер тестовых данных: {test_df.shape}")

# Анализ целевой переменной
print(f"\n Распределение целевой переменной (y):")
target_distribution = train_df['y'].value_counts(normalize=True)
print(target_distribution)
print(f"Дисбаланс классов: {target_distribution[1]:.3f} положительных")

# Проверка на пропущенные значения
print("\n Пропущенные значения в тренировочных данных:")
print(train_df.isnull().sum())

# Разделение на признаки и целевую переменную
X = train_df.drop('y', axis=1)
y = train_df['y']
X_test = test_df.copy()

test_ids = test_df['id']

# Определение типов признаков
numeric_features = ['age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
                   'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

categorical_features = ['job', 'marital', 'education', 'default', 'housing',
                       'loan', 'contact', 'month', 'day_of_week', 'poutcome']

print(f"\n📈 Числовые признаки ({len(numeric_features)}): {numeric_features}")
print(f"📊 Категориальные признаки ({len(categorical_features)}): {categorical_features}")

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
])


print("\n Создание модели логистической регрессии...")

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        class_weight='balanced',
        random_state=42,
        max_iter=1000,
        C=1.0
    ))
])

# Разделение на тренировочную и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n Размеры выборок:")
print(f"Тренировочная: {X_train.shape}")
print(f"Валидационная: {X_val.shape}")

# Обучение и оценка модели
print("\n Обучение логистической регрессии...")
model.fit(X_train, y_train)

# Предсказание на валидационной выборке
y_val_pred = model.predict_proba(X_val)[:, 1]
val_score = roc_auc_score(y_val, y_val_pred)

print(f" ROC-AUC на валидационной выборке: {val_score:.4f}")

# Кросс-валидация на всех данных
print(f"\n Кросс-валидация модели...")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc')
print(f"ROC-AUC при кросс-валидации: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Обучение финальной модели на всех данных
print(f"\n Обучение финальной модели на всех данных...")
final_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        class_weight='balanced',
        random_state=42,
        max_iter=1000,
        C=1.0
    ))
])

final_model.fit(X, y)

# Предсказание на тестовых данных
print("🔮 Прогнозирование на тестовых данных...")
test_predictions = final_model.predict_proba(X_test)[:, 1]

# Анализ предсказаний
print(f"\n📊 Статистика предсказаний:")
print(f"Минимальная вероятность: {test_predictions.min():.4f}")
print(f"Максимальная вероятность: {test_predictions.max():.4f}")
print(f"Средняя вероятность: {test_predictions.mean():.4f}")
print(f"Медианная вероятность: {np.median(test_predictions):.4f}")


submission = pd.DataFrame({
    'id': test_ids,
    'y': test_predictions  # Наши предсказания
})

# Сохранение результатов
submission_file = 'my_submission.csv'
submission.to_csv(submission_file, index=False)

print(f"\n Файл для отправки сохранен как: {submission_file}")
print(f" Размер файла: {submission.shape}")
print(f" Пример первых 10 предсказаний:")
print(submission.head(10))

print(f"\n Проверка формата файла:")
print(f"Колонки: {list(submission.columns)}")
print(f"Типы данных: {submission.dtypes}")
print(f"Диапазон вероятностей: [{submission['y'].min():.3f}, {submission['y'].max():.3f}]")

# Дополнительная информация о качестве модели
y_train_pred = final_model.predict_proba(X)[:, 1]
train_auc = roc_auc_score(y, y_train_pred)
print(f"\n📈 ROC-AUC на всех тренировочных данных: {train_auc:.4f}")

print("\n" + "="*50)
print(" ПРОГРАММА УСПЕШНО ЗАВЕРШЕНА!")
print("="*50)
print(f" Отправьте файл: {submission_file}")
print(f" Ожидаемый ROC-AUC: ~{val_score:.4f}")
print("="*50)

📊 Загрузка данных...

 Анализ данных:
Размер тренировочных данных: (28832, 21)
Размер тестовых данных: (12356, 20)

 Распределение целевой переменной (y):
y
0    0.944263
1    0.055737
Name: proportion, dtype: float64
Дисбаланс классов: 0.056 положительных

 Пропущенные значения в тренировочных данных:
id                0
age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

📈 Числовые признаки (9): ['age', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
📊 Категориальные признаки (10): ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'da